In [1]:
import numpy as np

def slog2(x):
    if(x <= 0.0):
        return 0.0
    else:
        return np.log2(x)

In [2]:
def load_em_prob_matrix(em_prob_path):
#     lines = open('em_prob.txt').readlines()
    lines = open(em_prob_path).readlines()
    emprob = {}
    
    for i in range(len(lines)):
        lines[i] = lines[i].rstrip('\n')
        lines[i] = lines[i].split()
    
    states = lines[0]
    labels = lines[1]
    lines = lines[2:]
    for i in range(len(states)):
        temp_dict_row = {}
        for j in range(len(labels)):
            temp_dict_row[labels[j]] = float(lines[i][j])
        emprob[states[i]] = temp_dict_row
        
    return emprob

In [3]:
def load_tr_prob_matrix(tr_prob_path):
#     lines = open('tr_prob.txt').readlines()
    lines = open(tr_prob_path).readlines()
    trprob = {}
    
    for i in range(len(lines)):
        lines[i] = lines[i].rstrip('\n')
        lines[i] = lines[i].split()
    
    states = lines[0]
    labels = lines[1]
    lines = lines[2:]
    
    for i in range(len(states)):
        temp_dict_row = {}
        for j in range(len(labels)):
            temp_dict_row[labels[j]] = float(lines[i][j])
        trprob[states[i]] = temp_dict_row
    
    return trprob

In [4]:
# def viterbi(obspace, stspace, inprobs, observations, emprobs, trprobs):
#     t1 = [[0 for x in range(len(stspace))] for y in range(len(observations)+1)]#initializing 2d matrix for trace
#     seq = ''#sequence of states most likely to be followed for observed state
#     prob = 1;#I haven't implemented log probability version, therefore prob is 1 and others multiplied on to it

#     #initializing initial probabilities
#     for i in range(len(stspace)):
#         t1[0][i] = inprobs[i]

#     #for length of observations joint probabilities of switching states for emission are calculated here
#     for i in range(1, len(observations)+1):
#         for j in range(len(stspace)):#for alpha and beta cross switching
#             for k in range(len(stspace)):
#                 t1[i][j] += t1[i-1][k]*trprobs[stspace[k]][stspace[j]]*emprobs[stspace[k]][observations[i-1]]
  
#     #finding index of last state by last observation
#     ind = t1[-1].index(max(t1[i]))
#     seq+=stspace[ind]
#     for i in range(len(observations)-1, 0, -1):
#         templ = []
#         for j in range(len(stspace)):#scanning switch probabilities from the last index 'ind' and appends it to matrix
#             templ.append(t1[i][j]*trprobs[stspace[j]][stspace[ind]]*emprobs[stspace[j]][observations[i]])
#             print('TRPROB:',trprobs[stspace[j]][stspace[ind]],'EMPROB:',emprobs[stspace[j]][observations[i]],'IND:',ind,"T:",t1[i][j])
#         maxval = max(templ)
#         prev_ind = ind
#         ind = templ.index(maxval)
#         seq+=stspace[ind]
#         prob*=(trprobs[stspace[j]][stspace[ind]]*emprobs[stspace[j]][observations[i]])#calculates emission and transmission probabilities for that state

#     return t1[1:], seq[::-1], prob    # return all together

In [10]:
def viterbi(obspace, stspace, inprobs, observations, emprobs, trprobs, calc_type='normal'):
    t1 = [[0 for x in range(len(stspace))] for y in range(len(observations)+1)]#initializing 2d matrix for trace
    seq = ''#sequence of states most likely to be followed for observed state
    if(calc_type == 'normal'):
        prob = 1 #I haven't implemented log probability version, therefore prob is 1 and others multiplied on to it
    elif(calc_type == 'log'):
        prob = 0
    
    #initializing initial probabilities
    for i in range(len(stspace)):
        t1[0][i] = inprobs[i]

    #for length of observations joint probabilities of switching states for emission are calculated here
    for i in range(1, len(observations)+1):
        for j in range(len(stspace)):#for alpha and beta cross switching
            for k in range(len(stspace)):
                t1[i][j] += t1[i-1][k]*trprobs[stspace[k]][stspace[j]]*emprobs[stspace[k]][observations[i-1]]
  
    #finding index of last state by last observation
#     print('DEBUG',t1[i])
    ind = t1[-1].index(max(t1[i]))
    seq+=stspace[ind]
    for i in range(len(observations)-1, -1, -1):
        templ = []
        for j in range(len(stspace)):#scanning switch probabilities from the last index 'ind' and appends it to matrix
            templ.append(t1[i][j]*trprobs[stspace[j]][stspace[ind]]*emprobs[stspace[j]][observations[i]])
#             if(calc_type == 'normal'):
#                 templ.append(t1[i][j]*trprobs[stspace[j]][stspace[ind]]*emprobs[stspace[j]][observations[i]])
#             elif(calc_type == 'log'):
#                 templ.append(np.log2(t1[i][j])+np.log2(trprobs[stspace[j]][stspace[ind]])+np.log2(emprobs[stspace[j]][observations[i]]))
        maxval = max(templ)
        prev_ind = ind
        ind = templ.index(maxval)
        seq+=stspace[ind]
        
        if(calc_type == 'normal'):
            prob*=(trprobs[stspace[prev_ind]][stspace[ind]]*emprobs[stspace[ind]][observations[i]])#calculates emission and transmission probabilities for that state
        elif(calc_type == 'log'):
            prob+=(np.log2(trprobs[stspace[prev_ind]][stspace[ind]]) + np.log2(emprobs[stspace[ind]][observations[i]]))#calculates emission and transmission probabilities for that state

#         print('TRPROB:',trprobs[stspace[ind]][stspace[prev_ind]],'EMPROB:',emprobs[stspace[prev_ind]][observations[i]],'IND:',ind,"T:",t1[i][j])

    if(calc_type == 'log'):
        for i in range(len(observations)+1):
            for j in range(len(stspace)):
                t1[i][j] = slog2(t1[i][j])

    return t1[1:], seq[::-1], prob    # return all together

In [6]:
def assignment3():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'GGCT'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='normal')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
assignment3()

['a', 'b']
[0.19000000000000003, 0.11000000000000001]
[0.07060000000000002, 0.027400000000000008]
[0.0071760000000000036, 0.008104000000000002]
[0.0008889600000000005, 0.0022598400000000004]
Sequence: bbbbb
Probability: 0.002361960000000001


In [7]:
def takehome8():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'GTCCATCCACTA'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='normal')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
takehome8()

['a', 'b']
[0.19000000000000003, 0.11000000000000001]
[0.020400000000000005, 0.0316]
[0.002784000000000001, 0.008736]
[0.0005126400000000002, 0.00238656]
[0.00023228160000000008, 0.0004500864]
[3.440793600000001e-05, 0.00012384614400000001]
[6.812098560000002e-06, 3.3782538240000004e-05]
[1.6265650176000003e-06, 9.189406310400001e-06]
[7.693515325440002e-07, 1.7191557365760003e-06]
[1.2081631002624002e-07, 4.7186556420096005e-07]
[2.5029434828390405e-08, 1.286118654345216e-07]
[1.158283384691098e-08, 2.4151313171349507e-08]
Sequence: bbbbbbbbbbbbb
Probability: 2.964832302162949e-08


In [11]:
def takehome9():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'CGCCTCGGGC'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='log')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
takehome9()

['a', 'b']
[-4.0588936890535683, -2.8365012677171206]
[-5.3569750419865629, -5.1791879227712796]
[-8.3693261451031002, -7.0216754855541321]
[-10.957039256840291, -8.8897999193725443]
[-13.169772697818527, -10.766074630447831]
[-15.177019659326669, -12.64498072050359]
[-16.24896334427515, -15.064520548777342]
[-17.55134920966568, -17.403867319955157]
[-18.939128119283293, -19.61400958386697]
[-22.139490233649035, -21.420100831740758]
Sequence: bbbbbbbbbbb
Probability: -21.229536879


In [12]:
def takehome10():
    emprobm = load_em_prob_matrix('em_prob_final.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob_final.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['1', '2', '3']#observation space
    stspace = ['a', 'b', 'c']#state space
    inprobs = [0.5, 0.5, 0.0]#initial probabilities
    observations = '112122'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='log')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
    print(emprobm)
    print(trprobm)
takehome10()

['a', 'b', 'c']
[-2.6780719051126378, -3.0, -3.4150374992788439]
[-4.4150374992788439, -4.8300749985576878, -5.1926450779423963]
[-6.6076825772212393, -6.4150374992788439, -7.0931094043914813]
[-8.2451124978365318, -8.5081469036703261, -8.9556058806415457]
[-10.37064337992039, -10.129635280416595, -10.830074998557688]
[-12.245112497836532, -11.852795075057772, -12.620621632928739]
Sequence: aabbbbb
Probability: -14.0
{'a': {'1': 0.5, '2': 0.25, '3': 0.25}, 'b': {'1': 0.25, '2': 0.5, '3': 0.25}, 'c': {'1': 0.0, '2': 0.0, '3': 0.0}}
{'a': {'a': 0.5, 'b': 0.25, 'c': 0.25}, 'b': {'a': 0.25, 'b': 0.5, 'c': 0.25}, 'c': {'a': 0.0, 'b': 0.0, 'c': 0.0}}
